In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Define the neural network model
class CardDetector(nn.Module):
    def __init__(self):
        super(CardDetector, self).__init__()
        # Define the convolutional layers
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # Define the pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # Define the fully connected layers
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 3)
    
    # Define the forward pass of the neural network
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the neural network model
model = CardDetector()

# Define the image transformations to be applied to the data
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize the images to 64x64 pixels
    transforms.ToTensor(),  # Convert the images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the pixel values to have mean=0.5 and std=0.5
])

# Load the training dataset and create a dataloader
train_dataset = datasets.ImageFolder("train/", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=392, shuffle=True)

# Load the validation dataset and create a dataloader
val_dataset = datasets.ImageFolder("val/", transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Define the loss function to be used during training
criterion = nn.CrossEntropyLoss()

# Define the optimizer to be used during training
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 10
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # Set the model to training mode
        model.train()

        # Zero out the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update the model parameters
        optimizer.step()

        # Print statistics
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    print(f"Epoch {epoch}/{num_epochs}, Batch {i}/{len(train_loader)}, Loss: {loss.item():.4f}, Training Accuracy: {accuracy:.2f}%")

    # Switch to evaluation mode and calculate accuracy on validation set
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {accuracy:.2f}%")

    with open('model_state.pt','wb') as f:
        torch.save(model.state_dict(),f)

#TODO:
#1:Increase the size of the dataset: A larger dataset may help the model learn more diverse patterns and improve its accuracy.

#2:Adjust the learning rate: The learning rate determines the step size at which the model is updated during training. If it is too high or too low, the model's accuracy may suffer. Experiment with different learning rates to find the optimal one.

#3:Add more layers to the model: A deeper network with more layers may help the model learn more complex features and improve its accuracy.

#4:Use pre-trained models: Transfer learning can be a useful technique for improving the accuracy of a model. You can use pre-trained models and fine-tune them for your specific task.

#5:Perform data augmentation: Data augmentation involves applying transformations to the existing data to create new examples. This can help the model learn more robust features and improve its accuracy.

#6:Regularization: Regularization techniques like dropout or weight decay can help prevent overfitting and improve the generalization performance of the model.

#7:Hyperparameter tuning: Experiment with different hyperparameters like batch size, number of epochs, optimizer, etc. to find the optimal set of hyperparameters for your model.


Epoch 0/10, Batch 0/1, Loss: 1.0599, Training Accuracy: 72.16%
Epoch 0/10, Validation Accuracy: 66.48%
Epoch 1/10, Batch 0/1, Loss: 0.8849, Training Accuracy: 66.48%
Epoch 1/10, Validation Accuracy: 66.48%
Epoch 2/10, Batch 0/1, Loss: 0.7996, Training Accuracy: 66.48%
Epoch 2/10, Validation Accuracy: 66.48%
Epoch 3/10, Batch 0/1, Loss: 0.8090, Training Accuracy: 66.48%
Epoch 3/10, Validation Accuracy: 66.48%
Epoch 4/10, Batch 0/1, Loss: 0.7856, Training Accuracy: 66.48%
Epoch 4/10, Validation Accuracy: 66.48%
Epoch 5/10, Batch 0/1, Loss: 0.7884, Training Accuracy: 66.48%
Epoch 5/10, Validation Accuracy: 66.48%
Epoch 6/10, Batch 0/1, Loss: 0.7886, Training Accuracy: 66.48%
Epoch 6/10, Validation Accuracy: 66.48%
Epoch 7/10, Batch 0/1, Loss: 0.7768, Training Accuracy: 66.48%
Epoch 7/10, Validation Accuracy: 66.48%
Epoch 8/10, Batch 0/1, Loss: 0.7633, Training Accuracy: 66.48%
Epoch 8/10, Validation Accuracy: 66.48%
Epoch 9/10, Batch 0/1, Loss: 0.7546, Training Accuracy: 66.48%
Epoch 9/10